In [9]:
from jetcam.csi_camera import CSICamera
# from jetcam.usb_camera import USBCamera

camera = CSICamera(width=640, height=480)

camera.running = True

In [10]:
# unobserve all callbacks from camera in case we are running this cell for second time
camera.unobserve_all()

In [6]:
import shutil

shutil.rmtree("pics")

In [7]:
import ipywidgets
import traitlets

from IPython.display import display
from jetcam.utils import bgr8_to_jpeg
from jupyter_clickable_image_widget import ClickableImageWidget

import cv2
import os

camera_widget = ClickableImageWidget(width=camera.width, height=camera.height)
snapshot_widget = ipywidgets.Image(width=camera.width, height=camera.height)
traitlets.dlink((camera, 'value'), (camera_widget, 'value'), transform=bgr8_to_jpeg)


index = 0

os.makedirs("pics", exist_ok=True)

def save_snapshot(_, content, msg):
    global index
    if content['event'] == 'click':
        # display saved snapshot
        snapshot = camera.value.copy()
        cv2.imwrite(f"pics/pic{index}.jpg", snapshot)
        index+=1
        snapshot_widget.value = bgr8_to_jpeg(snapshot)
        
camera_widget.on_msg(save_snapshot)

data_collection_widget = ipywidgets.HBox([
    camera_widget,
    snapshot_widget
])

display(data_collection_widget)


In [14]:
import threading
import time

# --- auto capture section ---
def auto_capture():
    os.makedirs("road", exist_ok=True)
    total_time = 2 * 60  # 2 minutes
    interval = 1         # 1 seconds
    count = total_time // interval
    print(f"Starting auto capture: {count} images every {interval} seconds...")
    
    for i in range(count):
        frame = camera.value.copy()
        path = f"road/road_{i:03d}.jpg"
        cv2.imwrite(path, frame)
        print(f"[{i+1}/{count}] Saved {path}")
        time.sleep(interval)
    
    print("Auto capture completed.")

# Run auto capture in a background thread so it won’t block the notebook UI
thread = threading.Thread(target=auto_capture, daemon=True)
thread.start()

Starting auto capture: 120 images every 1 seconds...
[1/120] Saved road/road_000.jpg
[2/120] Saved road/road_001.jpg
[3/120] Saved road/road_002.jpg
[4/120] Saved road/road_003.jpg
[5/120] Saved road/road_004.jpg
[6/120] Saved road/road_005.jpg
[7/120] Saved road/road_006.jpg
[8/120] Saved road/road_007.jpg
[9/120] Saved road/road_008.jpg
[10/120] Saved road/road_009.jpg
[11/120] Saved road/road_010.jpg
[12/120] Saved road/road_011.jpg
[13/120] Saved road/road_012.jpg
[14/120] Saved road/road_013.jpg
[15/120] Saved road/road_014.jpg
[16/120] Saved road/road_015.jpg
[17/120] Saved road/road_016.jpg
[18/120] Saved road/road_017.jpg
[19/120] Saved road/road_018.jpg
[20/120] Saved road/road_019.jpg
[21/120] Saved road/road_020.jpg
[22/120] Saved road/road_021.jpg
[23/120] Saved road/road_022.jpg
[24/120] Saved road/road_023.jpg
[25/120] Saved road/road_024.jpg
[26/120] Saved road/road_025.jpg
[27/120] Saved road/road_026.jpg
[28/120] Saved road/road_027.jpg
[29/120] Saved road/road_028.jpg

In [8]:
# Cleanly release resources
import gc

camera.running = False
camera.cap.release()
del camera

gc.collect()


Exception in thread Thread-4:
Traceback (most recent call last):
  File "/usr/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.6/dist-packages/jetcam-0.0.0-py3.6.egg/jetcam/camera.py", line 34, in _capture_frames
    self.value = self._read()
  File "/usr/local/lib/python3.6/dist-packages/jetcam-0.0.0-py3.6.egg/jetcam/csi_camera.py", line 40, in _read
    raise RuntimeError('Could not read image from camera')
RuntimeError: Could not read image from camera



20